# WRF-HRRR GRIB2 Tutorial

### This tutorial includes a brief introduction to WRF-HRRR data, as well as how to download and extract the necessary information for a given geographical point in the continential United States.
### <p>
###    <center> this is how to center <center>
# </p>

This tutorial includes:

 - [File Overview](#overview) 
 - [Downloading HRRR GRIB2](#download)
 - [Extracting data from GRIB2](#extraction)
 - [Additional Resources](#additional_resources)


### File Overview <a class="anchor" id="overview"></a>
---
The Weather Research and Forecasting (WRF) Model is a numerical weather prediction system developed by a collaboration of research organizations for simulating and forecasting weather conditions. It is an open-source model, allowing for users to modify the model to fit their applications. The High Resolution Rapid Refresh (HRRR) model is a derivative of the WRF model, providing a higher resolution of predictions generated every hour. The numerical weather observations of the HRRR model span across the entire continental United States, having the ability to not only forecast the weather, but also capturing rapidly evolving weather phenonma and severe weather predictions. The numerical weather measurements and predictions are stored in GRIdded Binary version 2 (GRIB2) file format, a highly processed and compressed file format popular for storing weather measurements. 



### Downloading HRRR GRIB2 <a class="anchor" id="download"></a>
---
NOTE: The command line examples apply to debian-based linux distributions. 
1. Prepare the system
    - Begin by installing the necessary packages into your environment
    
        ```
        sudo apt-get install pip;
        pip install ecmwflibs;
        conda install -c conda-forge pygrib
        pip install herbie-data;
        ```
2. Download the files
    - First, import the necessary libraries 
    


In [1]:
import pygrib 
from herbie import Herbie


- Set the desired date (yyyymmdd HH:MM) and directory to save

In [ ]:
date = "20210101 11:00"
save_dir = "/home/kaleb/Desktop"

- set the desired number of hours to predict in the future (0 for real-time)
- NOTE: precipitation and all other accumulation metric values are 0 for real-time. 

In [ ]:
pred_hours = 0
precip_pred_hours = 0

- load the values into Herbie objects

In [ ]:
herb_obj = Herbie(date, model="hrrr",
                  product="sfc", save_dir=save_dir,
                  verbose=True, fxx=pred_hours,
                  priority= ['pando', 'pando2', 'aws', 'nomads',
                                           'google', 'azure'],# ['pando', 'pando2', 'aws', 'nomads'],
                  overwrite=True)

precip_herb_obj = Herbie(date, model="hrrr",
                  product="sfc", save_dir=save_dir,
                  verbose=True, fxx=precip_pred_hours,
                  priority= ['pando', 'pando2', 'aws', 'nomads',
                                           'google', 'azure'],# ['pando', 'pando2', 'aws', 'nomads'],
                  overwrite=True)

- Now the objects can be segmented into parameters of choice using regex strings of the shortnames for the parameters and their levels. A list of GRIB2 shortnames can be found [here](https://codes.ecmwf.int/grib/param-db/?filter=grib2) and an interactive tutorial of how do use regex strings can be found [here](https://regexone.com/)
    - When segmenting the files into parameters of choice, one can either keep the data in memory or download the file. The entire file will all parameters can be downloaded with the command:

In [ ]:
herb_obj.download()
precip_herb_obj.download()


### Extracting From GRIB2 <a class="anchor" id="extraction"></a>
---
To avoid the WRF files taking up a large chunk of hard drive memory, they can be segmented into only the needed parameters and kept in memory. 

It should be noted that the regex strings can be used for both downloading and loading the data into an array, however for the purposes of this tutorial, the data will be loaded into an array. 

The 6 HRRR parameters used in this dataset are 2 meter temperature, 2 meter relative humidity, u component of wind (level 1000kPa), v component of wind (level 1000kPa), wind gust, and downward shortwave radiation flux. The steps to extracting the data for these parameters are as follows: 

- declare the geographic coordinates of interest with a longitude, latitude tuple, with longitude in the (-180, 180) range. 


In [ ]:
lsu = (-91.1783, 30.414)
ull = (-92.014, 30.213)
coordinates = [lsu, ull]
coord_names = ["lsu", "ull"]

- use regex strings to load the herbie object into a segmented xarray

In [ ]:
t2m_rh = herb_obj.xarray(":(?:TMP|RH):2 m")
u_v = herb_obj.xarray(":(U|V)GRD:1000 mb:")
dswrf = herb_obj.xarray(":DSWRF:surface:anl")
gust = herb_obj.xarray(":GUST:")

- use the xarrays and coordinates to generate an array of the associated values

In [ ]:
t2m_values = t2m_rh.herbie.nearest_points()



It should be noted that the `nearest_points` method is fragile. If too many points of latitude and longitude are passed, the time complexity of the method will skyrocket and in the worst case the code will break. 

### Additional Resources <a class="anchor" id="additional_resources"></a>
--- 

ECMWF homepage
Pygrib homepage
Herbie Documentation
NOAA homepage


